In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


#Potentially useful astropy stuff
import astropy
import astropy.io.ascii as ascii
from astropy.table import Table
from astropy.io import fits
from astropy.stats import LombScargle
from astropy.stats import sigma_clipped_stats
from astropy.stats import sigma_clip
from astropy.modeling import powerlaws
from astropy import constants as const
from astropy import units as u

#For reading in and organizing data
import pandas as pd
import requests
import json

#Misc
import scipy
from scipy import stats
from scipy.optimize import curve_fit
from collections import Counter
import itertools


#Making Figures look nice
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.size'] = 18

In [3]:
filenames = []

for i in range(1000):
    if i<10:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo20613500'
                         +str(i)+'-c03_llc.fits')
    elif i<100:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo2061350'
                         +str(i)+'-c03_llc.fits')
    else:
        filenames.append('https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135'
                         +str(i)+'-c03_llc.fits')


filenames

['https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135000-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135001-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135002-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135003-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135004-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135005-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135006-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135007-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135008-c03_llc.fits',
 'https://archive.stsci.edu/missions/k2/lightcurves/c3/206100000/35000/ktwo206135009-c03_llc.fits',


In [9]:
d = {}
for j, filename in enumerate(filenames):
    try:
        with fits.open(filename, mode="readonly") as hdulist:
            d["k2bjds{0}".format(j)] = hdulist[1].data['TIME'] 
            d["pdcsap_flux{0}".format(j)] = hdulist[1].data['PDCSAP_FLUX']
            d["pdcsap_flux_err{0}".format(j)] = hdulist[1].data['PDCSAP_FLUX_ERR']
    except: 
        print('no file for '+str(j))

no file for 0
no file for 1
no file for 2
no file for 3
no file for 4
no file for 5
no file for 6
no file for 7
no file for 8
no file for 9
no file for 10
no file for 12
no file for 13
no file for 14
no file for 15
no file for 16
no file for 17
no file for 18
no file for 19
no file for 20
no file for 21
no file for 22
no file for 23
no file for 24
no file for 25
no file for 26
no file for 27
no file for 28
no file for 29
no file for 30
no file for 31
no file for 33
no file for 34
no file for 35
no file for 36
no file for 37
no file for 38
no file for 39
no file for 40
no file for 41
no file for 42
no file for 43
no file for 44
no file for 45
no file for 46
no file for 47
no file for 48
no file for 49
no file for 50
no file for 51
no file for 52
no file for 53
no file for 54
no file for 55
no file for 56
no file for 57
no file for 58
no file for 59
no file for 60
no file for 62
no file for 63
no file for 64
no file for 65
no file for 66
no file for 67
no file for 68
no file for 69
no fi

no file for 545
no file for 546
no file for 547
no file for 548
no file for 549
no file for 550
no file for 551
no file for 552
no file for 553
no file for 554
no file for 555
no file for 557
no file for 558
no file for 559
no file for 560
no file for 561
no file for 563
no file for 564
no file for 566
no file for 567
no file for 568
no file for 569
no file for 570
no file for 571
no file for 572
no file for 573
no file for 574
no file for 575
no file for 576
no file for 577
no file for 578
no file for 579
no file for 580
no file for 581
no file for 582
no file for 583
no file for 584
no file for 585
no file for 586
no file for 587
no file for 588
no file for 589
no file for 590
no file for 592
no file for 593
no file for 594
no file for 595
no file for 596
no file for 597
no file for 598
no file for 599
no file for 600
no file for 601
no file for 602
no file for 603
no file for 604
no file for 605
no file for 607
no file for 608
no file for 609
no file for 610
no file for 611
no file 

In [21]:
d

{'k2bjds11': array([2144.10322285, 2144.12365529, 2144.14408753, ..., 2213.22381809,
        2213.24424969, 2213.26468129]),
 'k2bjds129': array([2144.10275376, 2144.123186  , 2144.14361805, ..., 2213.22314985,
        2213.24358154, 2213.26401323]),
 'k2bjds130': array([2144.10312752, 2144.12355991, 2144.14399211, ..., 2213.22366885,
        2213.24410047, 2213.26453209]),
 'k2bjds170': array([2144.10275121, 2144.12318346, 2144.14361551, ..., 2213.22314659,
        2213.24357828, 2213.26400997]),
 'k2bjds222': array([2144.10279773, 2144.12323   , 2144.14366206, ..., 2213.22320685,
        2213.24363853, 2213.26407022]),
 'k2bjds257': array([2144.10291709, 2144.1233494 , 2144.14378151, ..., 2213.2233699 ,
        2213.24380156, 2213.26423321]),
 'k2bjds267': array([2144.10293541, 2144.12336773, 2144.14379985, ..., 2213.22339537,
        2213.24382702, 2213.26425867]),
 'k2bjds314': array([2144.10313125, 2144.12356365, 2144.14399584, ..., 2213.22367454,
        2213.24410615, 2213.26453

In [22]:
len(d)/3

46.0

In [25]:
for i, filename in enumerate(d):

    print(filename, d[filename])
    flux = pdcsap_fluxes[(~np.isnan(pdcsap_fluxes)) & (~np.isnan(k2_bjds))][:500]
    fluxerr = pdcsap_fluxes_err[np.where(np.isnan(pdcsap_fluxes_err) == False)[0]][:500]
    bjd = k2_bjds[np.where((np.isnan(k2_bjds) == False) & (np.isnan(pdcsap_fluxes) == False))[0]][:500]

k2bjds11 [2144.10322285 2144.12365529 2144.14408753 ... 2213.22381809 2213.24424969
 2213.26468129]
pdcsap_flux11 [      nan 271578.3  271606.34 ... 271639.22 271649.88 271629.47]
pdcsap_flux_err11 [       nan 14.30751   14.314381  ... 14.645105  14.6487055 14.640937 ]
k2bjds32 [2144.10318186 2144.12361427 2144.14404649 ... 2213.22375294 2213.24418455
 2213.26461616]
pdcsap_flux32 [      nan 113359.94 113365.82 ... 113319.84 113344.6  113318.58]
pdcsap_flux_err32 [      nan 10.189867 10.193085 ... 10.512779 10.50667  10.502965]
k2bjds61 [2144.10274522 2144.12317746 2144.14360951 ... 2213.2231389  2213.24357059
 2213.26400229]
pdcsap_flux61 [      nan 3628.4277 3632.4104 ... 3613.3076 3605.9377 3599.849 ]
pdcsap_flux_err61 [      nan 4.0970945 4.1035666 ... 4.4588475 4.451832  4.4440403]
k2bjds75 [2144.10321626 2144.12364869 2144.14408092 ... 2213.22380749 2213.24423909
 2213.26467069]
pdcsap_flux75 [      nan 249660.97 249709.56 ... 249911.98 249921.62 249893.98]
pdcsap_flux_err75 [   